In [2]:
# Install Gradio (using -q for quiet mode)
!pip install gradio -q

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import gradio as gr
import warnings
import os

print("Environment setup complete, Google Drive mounted.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Environment setup complete, Google Drive mounted.


In [5]:
# --- Configuration and Data Loading ---
# Please modify this path according to your Google Drive
dataset_path = '/content/drive/MyDrive/HAR prepocessed dataset/COEN498-691_HAR_preprocessed_dataset.csv'

if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset file not found, please check the path: {dataset_path}")

df = pd.read_csv(dataset_path)

# 1. Prepare Data (consistent with original notebook)
from sklearn.model_selection import train_test_split

X = df.drop(['activity_id', 'participant_id'], axis=1)
y = df['activity_id']

# Split data (for training Scaler and Encoder)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 2. Preprocessing
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

encoder = OneHotEncoder(sparse_output=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))

# 3. Model Definition (consistent with original notebook architecture)
n_features = X_train_scaled.shape[1]
n_classes = y_train_encoded.shape[1]

model = Sequential()
# UserWarning: Do not pass an `input_shape`/`input_dim`...
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model.add(Dense(128, activation='relu', input_shape=(n_features,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

# TODO load pretrained model weights file
weights_path = '' # <--- Please adjust this path to your actual weights file (e.g., '/content/drive/MyDrive/my_model_weights.h5')
if os.path.exists(weights_path):
    try:
        model.load_weights(weights_path)
        print(f"Loaded pretrained model weights from {weights_path}")
    except Exception as e:
        print(f"Error loading weights from {weights_path}: {e}. Model will be trained from scratch.")
else:
    print(f"Pretrained weights file not found at {weights_path}. Model will be trained from scratch.")

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 4. Model Training (reduced epochs for quick demonstration)
print("Starting MLP model training (Epochs=10)...")
# Note: The original notebook used epochs=50 for high accuracy,
# we use 10 here for faster demonstration.
model.fit(X_train_scaled, y_train_encoded, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
print("Model training complete.")

# 5. Define Gradio Prediction Function and Label Mapping
activity_ids_order = encoder.categories_[0].tolist()
ACTIVITY_NAMES = {
    1: "sitting",
    2: "walking",
    3: "running",
    4: "lying"
}
LABEL_MAP = {i: ACTIVITY_NAMES.get(activity_ids_order[i], f"Activity ID {activity_ids_order[i]}")
             for i in range(len(activity_ids_order))}


def predict_activity(*features):
    """Gradio wrapper function for prediction."""
    try:
        # Convert input and scale (using scaler fitted during training)
        input_data = np.array(features, dtype=np.float32).reshape(1, -1)
        scaled_data = scaler.transform(input_data)

        # Predict
        predictions = model.predict(scaled_data, verbose=0)[0]

        # Decode and format output
        predicted_index = np.argmax(predictions)
        confidence = predictions[predicted_index]
        predicted_label = LABEL_MAP.get(predicted_index, f"Unknown Index {predicted_index}")

        output_text = (
            f"Predicted Activity: **{predicted_label}**\n"
            f"Confidence: {confidence:.4f}\n\n"
            "--- Raw Probability Distribution (Softmax) ---\n"
        )

        sorted_indices = np.argsort(predictions)[::-1]
        for i in sorted_indices:
            label_name = LABEL_MAP.get(i, f"Unknown Index {i}")
            output_text += f"{label_name}: {predictions[i]:.4f}\n"

        return output_text

    except Exception as e:
        return f"Error during prediction: {e}"

# 6. Configure Gradio Interface
feature_names = X.columns.tolist()
feature_inputs = [gr.Number(label=name, value=0.0) for name in feature_names]

iface = gr.Interface(
    fn=predict_activity,
    inputs=feature_inputs,
    outputs=gr.Markdown(label="Prediction Result"),
    title="Human Activity Recognition (HAR) Predictor",
    description="Enter 37 sensor feature values (**these should be standardized feature values**), and the model will predict the activity type.",
    live=False
)

# 7. Launch Gradio Application
print("\n--- Gradio Interface Launched ---")
iface.launch(share=True)

Pretrained weights file not found at . Model will be trained from scratch.
Starting MLP model training (Epochs=10)...
Model training complete.

--- Gradio Interface Launched ---
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2260cf7af9f8e0e1c6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')